<a href="https://colab.research.google.com/github/1028Luo/ML-STUDY-NOTES/blob/main/SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro

In [28]:
from google.cloud import bigquery
from google.colab import auth

auth.authenticate_user()
client = bigquery.Client("bigquery-public-data")
dataset_ref = client.dataset("openaq", project="bigquery-public-data")

# get dataset
dataset = client.get_dataset(dataset_ref)

In [29]:
# List all the tables in the "hacker_news" dataset
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset (there are four!)
for table in tables:
    print(table.table_id)


global_air_quality


In [16]:
# fetch table
table_ref = dataset_ref.table("global_air_quality")
table = client.get_table(table_ref)

In [17]:
# schema is the structure of the table
table.schema

# Each schema field is a column, and includs four things:
# Column name
# Data type
# The mode of the column ('NULLABLE' means that a column allows NULL values, and is the default)
# A description of the data in that column

[SchemaField('location', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('city', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('country', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('pollutant', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('value', 'FLOAT', 'NULLABLE', None, None, (), None),
 SchemaField('timestamp', 'TIMESTAMP', 'NULLABLE', None, None, (), None),
 SchemaField('unit', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('source_name', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('latitude', 'FLOAT', 'NULLABLE', None, None, (), None),
 SchemaField('longitude', 'FLOAT', 'NULLABLE', None, None, (), None),
 SchemaField('averaged_over_in_hours', 'FLOAT', 'NULLABLE', None, None, (), None),
 SchemaField('location_geom', 'GEOGRAPHY', 'NULLABLE', None, None, (), None)]

In [18]:
# Preview the first 5 lines, just like df.head()
client.list_rows(table, max_results=5).to_dataframe()



,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours,location_geom
0,"Borówiec, ul. Drapałka",Borówiec,PL,bc,0.85217,2022-04-28 07:00:00+00:00,µg/m³,GIOS,1.0,52.276794,17.074114,POINT(52.276794 1)
1,"Kraków, ul. Bulwarowa",Kraków,PL,bc,0.91284,2022-04-27 23:00:00+00:00,µg/m³,GIOS,1.0,50.069308,20.053492,POINT(50.069308 1)
2,"Płock, ul. Reja",Płock,PL,bc,1.41000,2022-03-30 04:00:00+00:00,µg/m³,GIOS,1.0,52.550938,19.709791,POINT(52.550938 1)
3,"Elbląg, ul. Bażyńskiego",Elbląg,PL,bc,0.33607,2022-05-03 13:00:00+00:00,µg/m³,GIOS,1.0,54.167847,19.410942,POINT(54.167847 1)
4,"Piastów, ul. Pułaskiego",Piastów,PL,bc,0.51000,2022-05-11 05:00:00+00:00,µg/m³,GIOS,1.0,52.191728,20.837489,POINT(52.191728 1)


In [ ]:
# Preview the first five entries in the "by" column of the "full" table
client.list_rows(table, selected_fields=table.schema[:1], max_results=5).to_dataframe()

# Select, From & Where

In [19]:
# forms a query

query = """
        SELECT city, country
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """

In [30]:
# submitting the query

ONE_MB = 1000*1000
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=ONE_MB)

query_job = client.query(query, job_config = safe_config)
us_cities = query_job.to_dataframe()
us_cities.head()

Forbidden: 403 POST https://bigquery.googleapis.com/bigquery/v2/projects/bigquery-public-data/jobs?prettyPrint=false: Access Denied: Project bigquery-public-data: User does not have bigquery.jobs.create permission in project bigquery-public-data.

Location: None
Job ID: 319c4f7b-e698-4c07-b912-66e80b6c4e47


# Group By, Having & Count & Order BY

In [ ]:
# count: returns count of num
# group by: merge column
# group by having: specifies a criteria
# order by:
# extract: extrate day from date
query = """
        SELECT Animal, COUNT(Id), EXTRACT(DAY from Date)
        FROM a_dataset
        GROUP BY Animal
        HAVING COUNT(id) > 10
        ORDER BY a_column
        """

# As & With

In [ ]:
# AS: rename column generated
# AS WITH: return a temporary table
query = """
        SELECT Animal, COUNT(Id) AS number
        FROM a_dataset
        GROUP BY Animal
        HAVING COUNT(id) > 10
        ORDER BY a_column
        """

query = """
        WITH Animals AS
        (
          SELECT ID, Name
          FROM a_dataset
          WHERE Years_old>5
        )
        SELECT ID
        FROM Seniors
        """

# Joining Data

In [ ]:
# joining data with inner join
query = """
        SELECT p.Name AS Pet_Name, o.Name AS Owner_Name,
        FROM `bigquery-public-data.pet_records.pets` AS P
        INNER JOIN `bigquery-public-data.pet_records.owners` AS O
            ON p.ID = o.Pet_ID
        """